# Cleaning Department of Buildings Complaints dataset

#### Authors:
- __Jeremy Ondov:__ [GitHub](https://github.com/ondovj)
- __Mahdi Shadkam-Farrokhi:__ [GitHub](https://github.com/Shaddyjr) 

This notebook is intended for cleaning the DOB dataset we'll be working with in a separate [notebook for EDA and modeling](./dob_complaints.ipynb).

## Source Material
- [Data Source](https://data.cityofnewyork.us/Housing-Development/DOB-Complaints-Received/eabe-havv)
- [Complaint Codes](https://www1.nyc.gov/assets/buildings/pdf/complaint_category.pdf)
- [Disposition Codes](https://www1.nyc.gov/assets/buildings/pdf/bis_complaint_disposition_codes.pdf)
- [Data Explains](https://docs.google.com/spreadsheets/d/10p0HLqinKbUrSjKaZC2E0ZTHDXgULT0K/edit#gid=1015257717)
- [American Community Survey](https://www.census.gov/programs-surveys/acs/data.html)

## Loading libraries and data

In [189]:
import pandas as pd
import googlemaps
import time
import math

Given the massive size of the dataset, we'll only pull a sample from the relevant observations.

After some outside research, we discovered the DOB website went live in 2009, which drastically altered the shape of the data after that point. Therefore, we will only select observations from 2009 onward, as these are much more applicable to current events.

In [191]:
target_size = 100_000 # desired sample size

In [33]:
data_file = "./datasets/DOB_Complaints_Received.csv"
chunk_size = 100_000 # number of lines used for each iterated read through file
skip = math.ceil(1_300_000 / target_size) # the sample rate. Every "skip"th observation is selected

dtypes = {
    'Complaint Number':"int64",
    'ZIP Code':"object",
    'Special District':"object",
    'Complaint Category':"object",
    'Unit':"object",
    'Date Entered':"object",
    'Status':"object",
    'House Street':"object",
    'House Number':"object"
}

keepers = [
    'Complaint Number',
    'ZIP Code',
    'Special District',
    'Complaint Category',
    'Unit',
    'Date Entered',
    'Inspection Date',
    'Status',
    'House Street',
    'House Number'
]

iteration_obj = pd.read_csv(
                    data_file, 
                    usecols = keepers, 
                    parse_dates=['Date Entered'], 
                    iterator = True,
                    chunksize = chunk_size,
                    dtype = dtypes,
                    index_col = 0
                );

df = None
current_n = 0
while df is None or (df.shape[0] < target_size and iteration_obj._currow <= 2_300_000):
    raw_dataframe = iteration_obj.get_chunk()
    # removing anything before 2009
    filtered_dataframe = raw_dataframe["2009" < raw_dataframe["Date Entered"]]
    if df is None:
        df = filtered_dataframe
    # adding to sample
    df = pd.concat([df, filtered_dataframe.iloc[::skip,:]], axis = 0)
    current_n += chunk_size
    print("Working up to row # {} | Current sample length = {}".format(current_n,df.shape[0]))

iteration_obj.close() # not sure if needed, but good practice to close connections

# clean up index
df.reset_index(inplace = True)

Working up to row # 100000 | Current sample length = 55256
Working up to row # 200000 | Current sample length = 59144
Working up to row # 300000 | Current sample length = 63052
Working up to row # 400000 | Current sample length = 66961
Working up to row # 500000 | Current sample length = 70857
Working up to row # 600000 | Current sample length = 74768
Working up to row # 700000 | Current sample length = 78658
Working up to row # 800000 | Current sample length = 82596
Working up to row # 900000 | Current sample length = 86505
Working up to row # 1000000 | Current sample length = 90410
Working up to row # 1100000 | Current sample length = 94332
Working up to row # 1200000 | Current sample length = 98241
Working up to row # 1300000 | Current sample length = 102161


In [34]:
df.shape

(102161, 10)

In [36]:
df.head()

,Complaint Number,Status,Date Entered,House Number,ZIP Code,House Street,Special District,Complaint Category,Unit,Inspection Date
0,2193181,CLOSED,2013-12-17,573,10458,EAST FORDHAM ROAD,,4B,SEP,12/24/2013
1,1265849,CLOSED,2009-09-21,429,10075,EAST 77 STREET,,73,MAN.,10/03/2009
2,1404696,CLOSED,2015-09-02,21,10011,WEST 8 STREET,,37,ERT,09/03/2015
3,2149422,CLOSED,2010-08-25,2075,10462,WALLACE AVENUE,,23,ERT,10/09/2010
4,3312533,CLOSED,2009-07-20,819,11220,59 STREET,,90,CITY,09/15/2009


We'll be working with roughly 100,000 obsevations

---
We are also bringing in a dataset of median household income for each zip code in New York. This data was sourced from the American Community Survey, using their 5-year estimates from 2017, and adjusted to 2017 inflation levels.

[source](https://www.census.gov/programs-surveys/acs/data.html)

In [65]:
income_db = pd.read_csv("./datasets/ACS_17_5YR_S1901_with_ann.csv", header=1)
income_db.head()

FileNotFoundError: [Errno 2] File ./datasets/ACS_17_5YR_S1901_with_ann.csv does not exist: './datasets/ACS_17_5YR_S1901_with_ann.csv'

## Cleaning DOB dataset

### Filter only closed complaints

The only relevant observations are cases that are currently closed.

In [37]:
df = df[df["Status"] == "CLOSED"]

### Converting Inspection date to datetime

In [38]:
df["Inspection Date"].head()

0    12/24/2013
1    10/03/2009
2    09/03/2015
3    10/09/2010
4    09/15/2009
Name: Inspection Date, dtype: object

Some dates are erroneous and out of bounds for conversion to date time. Although a few observations can me inferentially imputed manually, it is impractical to include such a tactic in the main workflow for many hundreds of thousands of observations. 

According to the [documentation](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#timeseries-timestamp-limits), the earliest valid time stamp for proper formating is `pd.Timestamp.min`. Also, we'll remove any dates prior to 2009, which serves as our hard cutoff for consideration, since the DOB website came online January of 2009 and shows a significant deviation from prior years.

In [39]:
# filtering invalid years
years = df["Inspection Date"].apply(lambda a:a.split("/")[2])
invalid_year_indeces = years[years.astype(int) < 2009].index
df.loc[invalid_year_indeces,:].head()

,Complaint Number,Status,Date Entered,House Number,ZIP Code,House Street,Special District,Complaint Category,Unit,Inspection Date
2915,4389173,CLOSED,2009-02-09,10-93,11101,JACKSON AVENUE,,31,QNS.,12/30/2008
5953,3291457,CLOSED,2009-01-06,315,11207,PENNSYLVANIA AVENUE,,05,BKLYN,11/26/2008
9488,3291632,CLOSED,2009-01-07,491,11208,EMERALD STREET,,55,BKLYN,11/20/2008
9975,1310016,CLOSED,2011-09-22,170,10012,MERCER STREET,,23,SCFLD,09/30/2000
11655,3399780,CLOSED,2012-01-01,2223,11226,CORTELYOU ROAD,,37,ERT,01/02/0212


In [40]:
# dropping invalid years
df.drop(index = invalid_year_indeces, inplace = True)
df["Inspection Date"] = df["Inspection Date"].astype('datetime64[ns]')
df.describe(include = "all")

,Complaint Number,Status,Date Entered,House Number,ZIP Code,House Street,Special District,Complaint Category,Unit,Inspection Date
count,1.002170e+05,100217,100217,100217,100217,100217,100217,100217,100217,100217
unique,NaN,1,3593,15221,204,7179,2,106,28,3570
top,NaN,CLOSED,2009-02-12 00:00:00,15,11419,BROADWAY,,45,QNS.,2017-12-05 00:00:00
freq,NaN,100217,77,256,1775,1106,99504,14694,19081,92
first,NaN,NaN,2009-01-02 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,2009-01-02 00:00:00
last,NaN,NaN,2018-11-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,2018-11-06 00:00:00
mean,3.276906e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,1.266253e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.245559e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,2.166787e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
df.shape

(100217, 10)

We now have over 100,000 cleaned observations.

### Cleaning Income Dataset

For the income dataset, we are only going to use two of the columns, the 'Id2' column which describes the zip code, and the first income column, labeled 'Households; Estimate; Median income (dollars)'. These will be formatted so they can easily be added to the main dataframe. Further examinations into the other available data may be used in further iterations of this project.

In [ ]:
income_db.info()

Looking at the datatypes, the zip code is currently an integer, though we would prefer it be a string, so that will be converted. For the income column, it is currently being interpreted as an object, so we need to determine what possible non-numeric characters are present and deal with them.

In [ ]:
inc_cols = {"Households; Estimate; Median income (dollars)": "med_inc_zip",
           "Id2": "zip_code"}
income_db = income_db.rename(mapper=inc_cols, axis=1)
income_db.head()

In [ ]:
# converting zip from int to str for proper manipulation
income_db["zip_code"] = income_db["zip_code"].astype("str")

# eliminating any extraneous symbols for income
income_db["med_inc_zip"] = income_db["med_inc_zip"].str.replace("\D+", "")

income_db.head()

# Feature Engineering

### Creating target variable
Our target is the number of days until a complaint's inspection date: 

$$\text{Inspection Date} - \text{Data Entered} = \text{Days until Inspection}$$

In [42]:
df["days_until_inspection"] = df["Inspection Date"] - df["Date Entered"]

In [43]:
df.describe()

,Complaint Number,days_until_inspection
count,1.002170e+05,100217
mean,3.276906e+06,69 days 15:30:23.726513
std,1.266253e+06,189 days 17:28:03.537128
min,1.245559e+06,-1460 days +00:00:00
25%,2.166787e+06,1 days 00:00:00
50%,3.479654e+06,12 days 00:00:00
75%,4.496623e+06,66 days 00:00:00
max,5.138147e+06,3402 days 00:00:00


Some complaints took a negative number of days, which we investigated. Sources at NYC Open Data claimed some complaints were found by inspectors and inspected immediately, and were only filed official at a later date. Since these "complaints" are impossible to predict for, we'll remove them from consideration. 

In [44]:
# extracting raw number of days
df["days_until_inspection"] = df["days_until_inspection"].map(lambda x:x.days)

In [45]:
df[df["days_until_inspection"] <= 0].shape

(18081, 11)

We'll be dropping about 18,000 observations.

In [46]:
df = df[df["days_until_inspection"] > 0]

### Fixing Special District

The "Special District" column has an emptry string category, which we'll change to "NOT SPECIAL".

In [47]:
df["Special District"].unique()

array(['   ', 'IBZ'], dtype=object)

In [48]:
df["Special District"] = df["Special District"].map(lambda x: x if x != '   ' else "NOT SPECIAL")

In [49]:
df["Special District"].unique()

array(['NOT SPECIAL', 'IBZ'], dtype=object)

### Renaming columns
For simplicity and consistency, we'll be using snakecase for column naming.

In [50]:
# removing spaces & forcing all to lowercase
df.columns = [col.lower().replace(" ", "_") for col in df.columns]

### Handling Zip Codes

Since there are some entries with missing or corrupted zip codes, we are going to use the contextual address information to impute the correct zip codes. First, we will concatenate the house number and street names. Then, we will add in the correct borough of the address, utilizing the isolated first number from the "bin" column. Then, the partial address can be sent to Google's Geocode API, which will return the full address including the zip code.

In [51]:
# removing whitespace around address info
strip_cols = ["zip_code", "house_street", "house_number"]
df[strip_cols] = df[strip_cols].apply(lambda col: col.str.strip())

In [52]:
# isolating entries with missing zips
zip_df = df[df["zip_code"].str.len() < 5].copy()

In [53]:
# creating borough mapper
borough_codes = {
    "1": "Manhattan",
    "2": "Bronx",
    "3": "Brooklyn",
    "4": "Queens",
    "5": "Staten Island"
}

In [54]:
# adding boroughs (first digit of complaint) to addresses with missing zips
zip_df["address"] = (zip_df["house_number"] + " " +
                     zip_df["house_street"] + ", " + 
                     zip_df["complaint_number"].apply(lambda x: str(x)[0]).map(borough_codes) + 
                     ", NY")
zip_df.head(2)

,complaint_number,status,date_entered,house_number,zip_code,house_street,special_district,complaint_category,unit,inspection_date,days_until_inspection,address
3964,2181043,CLOSED,2013-01-14,1,,ANGELAS PLACE,NOT SPECIAL,30,BRONX,2013-01-24,10,"1 ANGELAS PLACE, Bronx, NY"
24756,3402510,CLOSED,2012-02-03,104,,GLENWOOD ROAD,NOT SPECIAL,05,BKLYN,2012-04-19,76,"104 GLENWOOD ROAD, Brooklyn, NY"


#### Scraping Zip Codes - Google Geocoding

For the purposes of automation and future scaling, the `zip_finder` function will take in a dataframe containing addresses and request the zip codes from the Google Maps API ([get API key here](https://developers.google.com/maps/documentation/javascript/get-api-key)). We will begin by securely importing our API key.

_NOTE: In order to replicate our process, a file named `env.json` will need to be created at the top-level of the directory using the following JSON format:_

```json
{
    "API KEY": "YOUR_API_KEY_HERE"
}
```

In [55]:
# making var for api key
ENV = pd.read_json("./env.json", typ="series")
API_KEY = ENV["API KEY"]

# setting client with api key
gmap_client = googlemaps.client.Client(key=API_KEY)

#### Creating `zip_finder` function
The function we are going to build will simply take in a dataframe with the correct address column. The Google Maps service will take the partial address from the dataframe, and the response will be a JSON object that is converted to a dictionary with each part of the full address as a key:pair entry. This zip code will then be mapped back onto the dataframe's zip code column.

In [56]:
def zip_finder(df, address_col, zip_code_col):
    '''Fetches missing zip codes from dataframe'''
    # easy part - loop thru addresses
    # run geocode request for each
    for address in df[address_col]:
        print("fetching address: ", address)
        
        # gets full address
        full_addr = googlemaps.geocoding.geocode(client = gmap_client, address = address)

        # isolates just the zip from the full address
        for addr_dict in full_addr[0]["address_components"]:
            if addr_dict["types"] == ["postal_code"]:
                zip_code = addr_dict["short_name"]
        print("found zip: ", zip_code)
        
        # connecting found zip back to entry with this address
        df.loc[df.index[df[address_col] == address], zip_code_col] = zip_code
        
        # spacing requests to not exceed rate limit
        time.sleep(0.5)

In [57]:
# adding missing zip codes using google maps API
zip_finder(zip_df, "address", "zip_code")

fetching address:  1 ANGELAS PLACE, Bronx, NY
found zip:  10465
fetching address:  104 GLENWOOD ROAD, Brooklyn, NY
found zip:  11236
fetching address:  3031 VETERANS ROAD WEST, Staten Island, NY
found zip:  10309
fetching address:  38-84 RAILROAD AVENUE, Queens, NY
found zip:  11101
fetching address:  3031 VETERANS ROAD WEST, Staten Island, NY
found zip:  10309
fetching address:  60 COLLISTER STREET, Manhattan, NY
found zip:  10013


Now that our isolated zip code dataframe has been filled, we can assign those zip codes back onto the original entries in the main dataframe.

In [63]:
# assigning the located zips back to the original entries
df.loc[zip_df.index, "zip_code"] = zip_df["zip_code"]

### Joining Income Information

While we could use zip codes to predict the target, however we suspect an area's demographics, such as median income, will have a truer connection to predicting the target.

Now that all of our entries have zip codes, we can merge the income database using the zip codes as the common column.

In [ ]:
df = pd.merge(df,
         income_db[["zip_code", "med_inc_zip"]],
         how="left",
         on="zip_code")
db.head()

## Handling Nulls

In [26]:
db.isnull().sum()

complaint_number         0
status                   0
date_entered             0
house_number             0
zip_code                 0
house_street             0
special_district         0
complaint_category       0
unit                     0
inspection_date          0
days_until_inspection    0
dtype: int64

We're not finding nulls in our dataset, however, as part of our meta cleaning process, we will drop any null observations.

We must account for this in our process as we're assuming these values are missing completely at random, or MCAR.

In [27]:
db = db.dropna()

In [28]:
db.shape

(82136, 11)

In [29]:
db.head()

,complaint_number,status,date_entered,house_number,zip_code,house_street,special_district,complaint_category,unit,inspection_date,days_until_inspection
1,2193181,CLOSED,2013-12-17,573,10458,EAST FORDHAM ROAD,NOT SPECIAL,4B,SEP,2013-12-24,7
6,1265849,CLOSED,2009-09-21,429,10075,EAST 77 STREET,NOT SPECIAL,73,MAN.,2009-10-03,12
12,1404696,CLOSED,2015-09-02,21,10011,WEST 8 STREET,NOT SPECIAL,37,ERT,2015-09-03,1
13,2149422,CLOSED,2010-08-25,2075,10462,WALLACE AVENUE,NOT SPECIAL,23,ERT,2010-10-09,45
14,3312533,CLOSED,2009-07-20,819,11220,59 STREET,NOT SPECIAL,90,CITY,2009-09-15,57


# Save cleaned data

We now have a complete dataset, which will be used in a [separate notebook for EDA and modeling](./dob_complaints.ipynb).

In [30]:
# saving db
db.to_csv("./datasets/cleaned.csv")